# Preparing a labeled object dataset for training using fastdup V1.0

In [ ]:
# download fastdup
!pip install pip -U
!pip install fastdup
!pip install pandas
!pip install plotly
!pip install wurlitzer
%load_ext wurlitzer

In [4]:
import fastdup
import pandas as pd

# Get mini-coco dataset

In [ ]:
# download mini-coco
!gdown --id 1t_l9uyBPfxSEzcajTk4a1TaQXzeRm9hw
!unzip coco_minitrain_25k.zip
!gdown --fuzzy https://drive.google.com/file/d/1i12p23cXlqp1QrXjAD_vu467r4q67Mq9/view\?usp\=sharing -O coco_minitrain_25k/annotations/

# Load annotations
We will use a simple converter to convert the COCO format JSON annotaion file into the fastdup annotation dataframe. This converter is applicable to any dataset which uses COCO format.

In [5]:
coco_csv = 'coco_minitrain_25k/annotations/coco_minitrain2017.csv'
coco_annotations = pd.read_csv(coco_csv, header=None, names=['img_filename', 'bbox_x', 'bbox_y',
                                                             'bbox_w', 'bbox_h', 'label', 'ext'])

coco_annotations['split'] = 'train'  # Only train files were loaded
coco_annotations = coco_annotations.drop_duplicates()

In [6]:
coco_annotations.head(3)

,img_filename,bbox_x,bbox_y,bbox_w,bbox_h,label,ext,split
0,000000131075.jpg,20.23,55.98,313.49,326.50,tv,0,train
1,000000131075.jpg,176.90,381.12,286.20,136.63,laptop,0,train
2,000000131075.jpg,369.96,361.35,72.76,73.91,laptop,0,train


# Run fastdup

In [ ]:
# run fastdup with annotations
image_dir = 'coco_minitrain_25k/images/train2017/'
work_dir = 'fastdup_minicoco'

fd = fastdup.create(work_dir=work_dir, input_dir=image_dir)
fd.run(annotations=coco_annotations)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2023-03-01 13:42:36 [INFO] Going to loop over dir /tmp/tmpj2s38o3n.csv
2023-03-01 13:42:36 [INFO] Found total 183544 images to run on
2023-03-01 13:44:43 [ERROR] Error: found invalid bounding box for image coco_minitrain_25k/images/train2017/000000528201.jpg. Please check bounding box file 264 341 0 5
2023-03-01 13:47:19 [ERROR] Error: found invalid bounding box for image coco_minitrain_25k/images/train2017/000000402248.jpg. Please check bounding box file 14 241 3 0
2023-03-01 13:48:06 [ERROR] Error: found invalid bounding box for image coco_minitrain_25k/images/train2017/000000141426.jpg. Please check bounding box file 106 224 2 0
2023-03-01 14:03:48 [ERROR] Error: found invalid bounding box for image coco_minitrain_25k/images/train2017/000000564557.jpg. Please check bounding box file 30 271 5 0
2023-03-01 14:10:00 [ERROR] Error: found invalid bounding box for image coco_minitrain_25k/images/train2017/000000183

# Get class statistics

In [ ]:
fd.annotations()['label'].value_counts()

person           50336
chair             7870
car               7703
bottle            4330
cup               4154
                 ...  
toothbrush         309
bear               296
parking meter      183
toaster             46
hair drier          40
Name: label, Length: 80, dtype: int64

## Class distribution
The dataset contains 25k images and 183k objects, an average of 7.3 objects per image. 

Interestingly, we see a highly unbalanced class distribution, where all 80 coco classes are present here, but there is a strong balance towards the person class, that accounts for over 56k instances (30.6%). Car and Chair classes also contain over 8k instances each, while at the bottom of the list the toaster and hair drier classes contain as few as 40 instances. 

Using `Plotly` we get a useful interactive histogram. 

In [ ]:
import plotly.express as px
fig = px.histogram(coco_annotations, x="label")
fig.show()

# Findind duplicates
First we visualize the general lists of duplicates

In [ ]:
fd.vis.component_gallery(min_items=2)

Filtered & Sorted Galleries

In [ ]:
# sorting by largest objects
fd.vis.component_gallery(sort_by='area', min_items=2, max_width=900)

In [ ]:
# clusters with 'person' labels
fd.vis.component_gallery(sort_by='area', min_items=2, slice='person', max_width=900)

# And outliers

In [ ]:
# visualize outliers
fd.vis.outliers_gallery()

# Find size and shape issues
Objects come in various shapes and sizes, and some times objects might be incorrectly labeled or too small to be useful. We will now find the smallest, narrowest and widest objects, and asses their usefulness. 

In [ ]:
annot = fd.annotations()
annot['area'] = annot['bbox_w'] * annot['bbox_h']
annot['aspect'] = annot['bbox_w'] / annot['bbox_h']

In [ ]:
# Smallest 5% of objects:
smallest_objects = annot[annot['area'] < annot['area'].quantile(0.05)].sort_values(by=['area'])

# 5% of extreme aspect ratios
aspect_ratio_objects = annot[(annot['aspect'] < annot['aspect'].quantile(0.05))
                             | (annot['aspect'] > annot['aspect'].quantile(0.95))].sort_values(by=['aspect'])


In [ ]:
# let's see the smallest objects
smallest_objects.head(3)

,img_filename,bbox_x,bbox_y,bbox_w,bbox_h,label,ext,split,fastdup_id,error_code,is_valid,area,aspect
fd_index,,,,,,,,,,,,,
110320,000000205392.jpg,580.76,232.51,10.00,10.00,person,0,train,110320,VALID,True,100.0000,1.0
131657,000000320957.jpg,183.84,300.01,10.02,10.02,apple,0,train,131657,VALID,True,100.4004,1.0
21376,000000407413.jpg,301.32,175.41,10.06,10.06,person,0,train,21376,VALID,True,101.2036,1.0


In [ ]:
aspect_ratio_objects.head(3)

,img_filename,bbox_x,bbox_y,bbox_w,bbox_h,label,ext,split,fastdup_id,error_code,is_valid,area,aspect
fd_index,,,,,,,,,,,,,
134867,000000093298.jpg,230.69,1.11,16.64,419.24,kite,0,train,134867,VALID,True,6976.1536,0.039691
3642,000000002444.jpg,1.92,136.50,11.51,263.87,person,0,train,3642,VALID,True,3037.1437,0.043620
164216,000000116502.jpg,222.84,63.49,16.19,318.20,spoon,0,train,164216,VALID,True,5151.6580,0.050880


In [ ]:
aspect_ratio_objects.tail(3)

,img_filename,bbox_x,bbox_y,bbox_w,bbox_h,label,ext,split,fastdup_id,error_code,is_valid,area,aspect
fd_index,,,,,,,,,,,,,
4261,000000527098.jpg,33.00,216.00,602.00,18.00,boat,0,train,4261,VALID,True,10836.0000,33.444444
77850,000000444692.jpg,1.12,58.60,638.88,15.74,bench,0,train,77850,VALID,True,10055.9712,40.589581
173218,000000516740.jpg,17.27,197.91,601.64,13.43,train,0,train,173218,VALID,True,8080.0252,44.798213


Look at that! The slices reveal many items that are either tiny (10x10 pixels) or have extreme aspect ratios - as extreme at 1:45 - an object 601 pixels wide by only 13 pixels high. 

### Objects that didn't make the cut:
Let's look at objects deemed invalid by fastdup. These are either objects that are too small to be useful in our analysis (smaller than 10px), have bouding boxes with illeagal values (negative or beyond image boundaries), or are part of images that are missing. We can tell which is which by the `error_code` column in our dataframe.

In [ ]:
fd.invalid_instances().head(3)

,img_filename,bbox_x,bbox_y,bbox_w,bbox_h,label,ext,split,fastdup_id,error_code,is_valid
0,000000262162.jpg,437.17,244.79,19.52,9.93,mouse,0,train,16,ERROR_BAD_BOUNDING_BOX,False
1,000000524325.jpg,137.84,332.22,8.92,11.50,person,0,train,60,ERROR_BAD_BOUNDING_BOX,False
2,000000524325.jpg,177.35,294.13,5.32,11.92,person,0,train,65,ERROR_BAD_BOUNDING_BOX,False


#### Distribution of error codes:
A simple `value_counts` will tell us the distribution of the errors. We have found 18,592 (!) bounding boxes that are either too small or go beyond image boundaries. This is 10% of the data! Filtering them would both save us grusome debugging of training errors and failures and help up provide the model with useful size objects. 

In [ ]:
fd.invalid_instances()['error_code'].value_counts()

ERROR_BAD_BOUNDING_BOX    18592
Name: error_code, dtype: int64

# Find possible mis-labels
The fastdup similarity search and gallery is a strong tool for finding objects that are possibly mislabeled. By finding each object's nearest neighbors and their classes, we can find objects with classes contradicting their neighbors' - a strong sign for mislabels.

In [ ]:
fd.vis.similarity_gallery(label_col='label', num_images=25)